In [1]:
#导包
from netCDF4 import Dataset
from tensorflow.keras import layers, models
from tensorflow.keras.layers import ConvLSTM2D, BatchNormalization, LayerNormalization,Input, Conv3D, TimeDistributed, Flatten
from tensorflow.keras.layers import Concatenate,Conv2D,TimeDistributed, MaxPooling2D, Input, MaxPooling3D
from tensorflow.keras.layers import  Reshape,multiply
from tensorflow.keras.layers import Layer,Lambda,Dot,ReLU,LSTM, Dense, Dropout, Activation, Flatten,Attention
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential,Model,load_model
from tensorflow.keras.optimizers import Adam,SGD
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras.losses import MeanSquaredError
from sklearn.metrics import r2_score,accuracy_score,precision_score
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras import regularizers
import tensorflow as tf
from datetime import datetime
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from tensorflow.keras import models, layers
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
import tensorflow.keras as keras
from tensorflow.keras import layers, Sequential, regularizers
# calculate RMSE
from sklearn.metrics import mean_squared_error
from math import sqrt
import pandas as pd 
import tensorflow as tf
import gc 
import gzip
import matplotlib.pyplot as plt
import numpy as np
import netCDF4 as nc
import match
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import os 
%matplotlib inline

In [2]:
# 检查可用GPU数量
len(tf.config.experimental.list_physical_devices('GPU'))

1

# 读取数据

## SSH

In [3]:
data_ssh = nc.Dataset(r'D:/codeFile/jupyterDemo/3d_ts/BPNN/data/SSH/SSH_2005-2015_month.nc')

In [4]:
# 确认时间
time_ssh = data_ssh['time'][:]
print(len(time_ssh))#查看时间长度为132

132


In [6]:
# 查看研究区域经纬度
ssh_lat = data_ssh['lat'][:].data
#print(ssh_lat[:])
ssh_lon = data_ssh['lon'][:].data
#print(ssh_lon[:])

In [6]:
# 读取ssh
ssh = data_ssh['ssh'][:].data
print(ssh.shape)

(132, 280, 600)


In [7]:
# 查看ssh最大值最小值，看是否有异常值
np.nanmax(ssh),np.nanmin(ssh)

(2.20051, -0.43673334)

## SSS

In [8]:
data_sss = nc.Dataset(r'D:/codeFile/jupyterDemo/3d_ts/BPNN/data/SSS/dataset-sss-ssd-rep-monthly_2005-2020.nc')

In [9]:
# 确定时间
time_sss = data_sss['time'][:132]
print(len(time_sss))

132


In [10]:
# 获取研究位置经纬度
sss_lat = data_sss['lat'][:].data
#print(sss_lat[:])
sss_lon = data_sss['lon'][:].data
#print(sss_lon[:])

In [11]:
sss = data_sss['sos'][:132].data 
print(sss.shape)
sss = np.squeeze(sss) # 移除大小为一的维度
print(sss.shape)

(132, 1, 280, 600)
(132, 280, 600)


In [12]:
np.nanmax(sss),np.nanmin(sss)

(36.072178, 13.604756)

## SSW

In [13]:
data_ssw = nc.Dataset(r'D:/codeFile/jupyterDemo/3d_ts/BPNN/data/SSW/anon_ssw_2005_2018_Month_data.nc')

In [14]:
# 确定时间
time_ssw = data_ssw['time'][:132]
print(len(time_ssw))

132


In [15]:
lat_ssw = data_ssw['lat'][:].data
#print(lat_ssw[314:594])
lon_ssw = data_ssw['lon'][:].data
#print(lon_ssw[440:1040])

# lat 0.125    ----    69.875
# lon  110.125    ----    259.875

In [16]:
# 获取U分量 和 V分量     time   lat    lon
uwnd = data_ssw['uwnd'][:132,314:594,440:1040].data  
vwnd = data_ssw['vwnd'][:132,314:594,440:1040].data

In [17]:
print(uwnd.shape)
print(vwnd.shape)

(132, 280, 600)
(132, 280, 600)


In [18]:
np.nanmax(uwnd),np.nanmin(uwnd),np.nanmax(vwnd),np.nanmin(vwnd)

(12.433594, -11.7988205, 8.8359995, -13.645481)

## 3DT

In [19]:
data_3dt_1 = nc.Dataset(r'D:/codeFile/jupyterDemo/3d_ts/BPNN/data/3DT/dataset-armor-3d-rep-monthly_2005-2009.nc')
data_3dt_2 = nc.Dataset(r'D:/codeFile/jupyterDemo/3d_ts/BPNN/data/3DT/dataset-armor-3d-rep-monthly_2010-2015.nc')

In [20]:
sst_1 = data_3dt_1['to'][:].data
sst_2 = data_3dt_2['to'][:].data

In [21]:
print(sst_1.shape)
print(sst_2.shape)

(60, 43, 280, 600)
(72, 43, 280, 600)


###  0 - 2000m 温度数据

In [22]:
# 将sst_1 和 sst_2 合并
temp_depth = np.append(sst_1,sst_2,axis=0)
print(temp_depth.shape)

(132, 43, 280, 600)


In [23]:
# 所有深度层
depths = [0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,80,90,100,125,150,175,200,225,250,275,300,350,400,
         450,500,550,600,700,800,900,1000,1100,1200,1300,1400,1500,1750,2000]

#选取的深度层
depths_use = [10,15,20,30,40,50,60,70,80,90,100,125,150,175,200,225,250,275,300,350,400,
              500,600,700,800,900,1000,1100,1300,1500,1750,2000]

#[50,100,125,150,200,250,300,400,500,600,700,800,900,1000]

In [24]:
temp_depth_use = temp_depth[:,1:2]

In [25]:
# 获取选取的深度层的所有信息
for i,depth in enumerate(depths):
    if(depth in depths_use):
        #print(depths[i])
        temp_depth_use = np.concatenate((temp_depth_use,temp_depth[:,i:i+1]),axis=1)    

In [26]:
print(temp_depth_use.shape)
# 将深度维度放到最后
temp_depth_use = np.transpose(temp_depth_use, (0,2,3,1))
print(temp_depth_use.shape)

(132, 33, 280, 600)
(132, 280, 600, 33)


## 表面温度数据

In [27]:
sst = temp_depth[:,0:1,:,:]

In [28]:
sst = np.squeeze(sst)
print(sst.shape)

(132, 280, 600)


## 数据合并

In [29]:
# 将所有表层数据在通道维度进行拼接
print(ssh.shape,sss.shape,uwnd.shape,vwnd.shape,sst.shape)
cat_data =np.stack((ssh,sss,uwnd,vwnd,sst),axis=3)
cat_data.shape

(132, 280, 600) (132, 280, 600) (132, 280, 600) (132, 280, 600) (132, 280, 600)


(132, 280, 600, 5)

In [30]:
# 将表面数据和水下数据在最后一个维度拼接，将空值删除。
cat_data_ss_depth = np.concatenate((cat_data,temp_depth_use),axis=-1)

In [31]:
cat_data_ss_depth.shape   # 5+33=38

(132, 280, 600, 38)

## 将两个温度异常区域的数据设置为nan

In [32]:
test = temp_depth_use[:,:,:,27:28]
test[test < 0.4] = np.nan
test[test > 10] = np.nan

In [33]:
cat_data_ss_depth[:,:,:,32:33] = test

# 验证集和测试集的划分

In [35]:
# 最后一年的数据作为test
test_num = 120
train = cat_data_ss_depth[:test_num]
test = cat_data_ss_depth[test_num:]

In [36]:
train.shape,test.shape

((120, 280, 600, 38), (12, 280, 600, 38))

## 去掉合并数据中的nan


In [37]:
train.shape 

(120, 280, 600, 38)

In [38]:
# 将训练数据展平，因为简单的全连接只涉及数据与数据之间的关系，没涉及到时间和空间。当前就相当于使用便面数据的5个数去反演深度数据的1个数。
train_flatten = train.reshape(-1,38)
#将默认填充值设置为nan
train_flatten[train_flatten == 32767] = np.nan
print('train删除nan值之前',train_flatten.shape)
# 使用 numpy.isnan 函数检测NaN值
nan_mask = np.isnan(train_flatten)
# 通过布尔索引删除NaN值
train_without_nan = train_flatten[~nan_mask.any(axis=1)]
print('train删除nan值之后',train_without_nan.shape)

train删除nan值之前 (20160000, 38)
train删除nan值之后 (12427525, 38)


In [39]:
test_flatten =  test.reshape(-1,38)
test_flatten[test_flatten == 32767] = np.nan
print('test删除nan值之前',test_flatten.shape)
# 使用 numpy.isnan 函数检测NaN值
nan_mask = np.isnan(test_flatten)
# 通过布尔索引删除NaN值
test_without_nan = test_flatten[~nan_mask.any(axis=1)]
print('test删除nan值之后',test_without_nan.shape)

test删除nan值之前 (2016000, 38)
test删除nan值之后 (1242592, 38)


## 划分训练数据和目标数据

In [17]:
# 将表面数据和次表层数据分开
x_test = test_without_nan[:,:5]
y_test = test_without_nan[:,5:]

x_train_val = train_without_nan[:,:5]
y_train_val = train_without_nan[:,5:]

NameError: name 'test_without_nan' is not defined

In [41]:
print('x_test.shape:',x_test.shape)
print('y_test.shape',y_test.shape)
print('x_train_val',x_train_val.shape)
print('y_train_val',y_train_val.shape)

x_test.shape: (1242592, 5)
y_test.shape (1242592, 33)
x_train_val (12427525, 5)
y_train_val (12427525, 33)


In [42]:
#划分验证集 和 测试集
x_train, x_val, y_train, y_val = train_test_split(x_train_val, y_train_val, test_size=0.25, random_state=10)

In [43]:
#查看其形状
x_train.shape,x_test.shape,x_val.shape,y_train.shape,y_test.shape,y_val.shape

((9320643, 5),
 (1242592, 5),
 (3106882, 5),
 (9320643, 33),
 (1242592, 33),
 (3106882, 33))

# 特征归一化

In [ ]:
# 记录数据集的形状
# x_train_shape = x_train.shape
# y_train_shape = y_train.shape

# x_test_shape = x_test.shape
# y_test_shape = y_test.shape

# x_val_shape = x_val.shape
# y_val_shape = y_val.shape

In [44]:
# 归一化的作用：消除不同特征之间的量纲影响。
scaler_f = StandardScaler()
x_train = scaler_f.fit_transform(x_train) 
x_test = scaler_f.transform(x_test)
x_val = scaler_f.transform(x_val)

scaler_l = StandardScaler()
y_train = scaler_l.fit_transform(y_train)
y_test = scaler_l.transform(y_test)
y_val = scaler_l.transform(y_val)

In [ ]:
# x_train = np.reshape(x_train,(x_train_shape))
# x_test = np.reshape(x_test,(x_test_shape))
# x_val = np.reshape(x_val,(x_val_shape))
# y_train = np.reshape(y_train,(y_train_shape))
# y_test = np.reshape(y_test,(y_test_shape))
# y_val = np.reshape(y_val,(y_val_shape))

In [45]:
# 特征归一化后没有改变数据形状
x_train.shape,x_test.shape,x_val.shape,y_train.shape,y_test.shape,y_val.shape

((9320643, 5),
 (1242592, 5),
 (3106882, 5),
 (9320643, 33),
 (1242592, 33),
 (3106882, 33))

# 预聚类（不用管）

In [ ]:
# 一下部分为测试预聚类方法， 没有得到较好的结果，先不使用。

In [ ]:
# import numpy as np
# from sklearn.cluster import KMeans

# def run_kmeans(data, n_clusters, n_runs):
#     best_score = None
#     best_labels = None
    
#     for _ in range(n_runs):
#         kmeans = KMeans(n_clusters=n_clusters)
#         labels = kmeans.fit_predict(data)
#         score = kmeans.score(data)
        
#         if best_score is None or score < best_score:
#             best_score = score
#             best_labels = labels
    
#     return best_labels

In [ ]:
# n_clusters = 28
# n_runs = 20
# labels = run_kmeans(x_train, n_clusters, n_runs)

# print("聚类结果：")
# print(labels)

In [ ]:
# #np.savetxt('labels.txt',labels)
# b =  np.loadtxt("labels.txt", delimiter=',')  # 读取文件

## 根据聚类结果排序

In [ ]:
# # 将数据和对应的聚类标签合并为一个数组
# data_with_labels = np.column_stack((x_train, labels))
# # 按照聚类标签对数据进行排序
# sorted_data = data_with_labels[data_with_labels[:, -1].argsort()]
# sorted_data_x = sorted_data[:,:5]

In [ ]:
# # 将数据和对应的聚类标签合并为一个数组
# data_with_labels = np.column_stack((y_train, labels))
# # 按照聚类标签对数据进行排序
# sorted_data = data_with_labels[data_with_labels[:, -1].argsort()]
# sorted_data_y = sorted_data[:,:15]

In [ ]:
# sorted_data_x.shape,sorted_data_y.shape 

# 定义网络结构

In [46]:
# 创建神经网络模型
def create_model():   
    model = models.Sequential([
        layers.Dense(200, activation='tanh', input_shape=(5,)),  # 输入层和隐藏层
        layers.Dense(15, activation='tanh'),
        layers.Dense(8, activation='tanh'),
        layers.Dense(1, activation='linear')  # 输出层，num_depths表示深度的数量
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model 
#model.summary()


# 若自定义损失函数


# # 定义自定义损失函数
# def my_custom_loss(y_true, y_pred):  # 参数位置不能随意交换，第一个参数必须是真实值，第二个参数必须是预测值
#     # 例如，你可以计算平方误差并添加一些自定义的正则化或其他惩罚
#     loss = tf.reduce_mean(tf.square(y_true - y_pred))  # 计算均方误差
#     return loss

# # 在编译模型时使用自定义损失函数
# model.compile(optimizer='adam', loss=my_custom_loss)

## 网格搜索交叉验证寻找最佳参数

In [ ]:
# model = KerasRegressor(build_fn=create_model,verbose=2)

In [ ]:
# param_grid = {
#     'neurons': [4,8,15,30,50,100,150,200],
# }

In [ ]:
# # 使用GridSearchCV进行交叉验证并搜索最佳参数组合：
# grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
# grid_result = grid_search.fit(x_train, y_train[:,14:15],epochs=200,verbose=2,batch_size=1024)

In [ ]:
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

## 编译模型

In [47]:
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 200)               1200      
_________________________________________________________________
dense_1 (Dense)              (None, 15)                3015      
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 128       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 9         
Total params: 4,352
Trainable params: 4,352
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# # 创建神经网络模型
# num_depths = 1
# num_features = 5
# # # 编译模型
# # model.compile(optimizer='adam', loss='mean_squared_error')
# checkpoint_path='D:/codeFile/jupyterDemo/3d_ts/BPNN/model/new_0-2000m_24.6.6/BPNN/temp/bpnn_model.h5'
# keras_callbacks   = [
#       EarlyStopping(monitor='val_loss', patience=30, mode='min', min_delta=0.001),
#       ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True, mode='min')
# ]

## 模型训练

### 对指定层进行训练

In [ ]:
# num_epochs = 1000
# batch_size = 1024

# history = model.fit(sorted_data_x, sorted_data_y[:,14:], validation_data=(x_val,y_val[:,14:]) ,
#                     epochs=num_epochs, batch_size=batch_size, verbose=2, callbacks=keras_callbacks)

### 对所有层进行训练

In [49]:
def BPNN_model(x_train,y_train,x_val,y_val,name):
    # 创建网络
    model = create_model()
    # 编译网络
    model.compile(optimizer='adam', loss='mean_squared_error')
    # 设置早停 和 模型保存路径
    checkpoint_path=name
    keras_callbacks   = [
          EarlyStopping(monitor='val_loss', patience=30, mode='min', min_delta=0.001),
          ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True, mode='min')
    ]
    # 模型训练
    model.fit(x_train, y_train, validation_data=(x_val,y_val),
        epochs=1000, batch_size=10240, verbose=2, callbacks=keras_callbacks)

In [52]:
#对所有层进行建模
depth=[5,10,15,20,30,40,50,60,70,80,90,100,125,150,175,200,225,250,275,300,350,400,500,600,700,800,900,1000,1100,1300,1500,1750,2000]
for i in range(0,33): #0-14
    BPNN_model(x_train, y_train[:,i:i+1],x_val, y_val[:,i:i+1],
                    'D:/codeFile/jupyterDemo/3d_ts/BPNN/model/new_0-2000m_24.6.6/BPNN/temp/model1/bpnn_temp_'+str(depth[i])+'m.h5')

Epoch 1/1000
911/911 - 2s - loss: 0.0131 - val_loss: 0.0013
Epoch 2/1000
911/911 - 1s - loss: 0.0012 - val_loss: 0.0012
Epoch 3/1000
911/911 - 1s - loss: 0.0012 - val_loss: 0.0012
Epoch 4/1000
911/911 - 1s - loss: 0.0012 - val_loss: 0.0012
Epoch 5/1000
911/911 - 1s - loss: 0.0012 - val_loss: 0.0012
Epoch 6/1000
911/911 - 1s - loss: 0.0012 - val_loss: 0.0012
Epoch 7/1000
911/911 - 1s - loss: 0.0012 - val_loss: 0.0012
Epoch 8/1000
911/911 - 1s - loss: 0.0012 - val_loss: 0.0012
Epoch 9/1000
911/911 - 1s - loss: 0.0012 - val_loss: 0.0012
Epoch 10/1000
911/911 - 1s - loss: 0.0012 - val_loss: 0.0012
Epoch 11/1000
911/911 - 1s - loss: 0.0012 - val_loss: 0.0012
Epoch 12/1000
911/911 - 1s - loss: 0.0012 - val_loss: 0.0012
Epoch 13/1000
911/911 - 1s - loss: 0.0012 - val_loss: 0.0012
Epoch 14/1000
911/911 - 1s - loss: 0.0012 - val_loss: 0.0012
Epoch 15/1000
911/911 - 1s - loss: 0.0012 - val_loss: 0.0012
Epoch 16/1000
911/911 - 1s - loss: 0.0012 - val_loss: 0.0012
Epoch 17/1000
911/911 - 1s - loss

911/911 - 1s - loss: 0.0079 - val_loss: 0.0079
Epoch 13/1000
911/911 - 1s - loss: 0.0079 - val_loss: 0.0078
Epoch 14/1000
911/911 - 1s - loss: 0.0078 - val_loss: 0.0078
Epoch 15/1000
911/911 - 1s - loss: 0.0078 - val_loss: 0.0078
Epoch 16/1000
911/911 - 1s - loss: 0.0077 - val_loss: 0.0077
Epoch 17/1000
911/911 - 1s - loss: 0.0077 - val_loss: 0.0077
Epoch 18/1000
911/911 - 1s - loss: 0.0077 - val_loss: 0.0076
Epoch 19/1000
911/911 - 1s - loss: 0.0076 - val_loss: 0.0077
Epoch 20/1000
911/911 - 1s - loss: 0.0076 - val_loss: 0.0076
Epoch 21/1000
911/911 - 1s - loss: 0.0076 - val_loss: 0.0075
Epoch 22/1000
911/911 - 1s - loss: 0.0076 - val_loss: 0.0075
Epoch 23/1000
911/911 - 1s - loss: 0.0075 - val_loss: 0.0075
Epoch 24/1000
911/911 - 1s - loss: 0.0075 - val_loss: 0.0075
Epoch 25/1000
911/911 - 1s - loss: 0.0075 - val_loss: 0.0075
Epoch 26/1000
911/911 - 1s - loss: 0.0075 - val_loss: 0.0074
Epoch 27/1000
911/911 - 1s - loss: 0.0074 - val_loss: 0.0074
Epoch 28/1000
911/911 - 1s - loss: 0.0

Epoch 57/1000
911/911 - 1s - loss: 0.0146 - val_loss: 0.0147
Epoch 58/1000
911/911 - 1s - loss: 0.0146 - val_loss: 0.0146
Epoch 59/1000
911/911 - 1s - loss: 0.0146 - val_loss: 0.0145
Epoch 1/1000
911/911 - 2s - loss: 0.0490 - val_loss: 0.0293
Epoch 2/1000
911/911 - 1s - loss: 0.0261 - val_loss: 0.0243
Epoch 3/1000
911/911 - 1s - loss: 0.0234 - val_loss: 0.0227
Epoch 4/1000
911/911 - 1s - loss: 0.0222 - val_loss: 0.0217
Epoch 5/1000
911/911 - 1s - loss: 0.0213 - val_loss: 0.0209
Epoch 6/1000
911/911 - 1s - loss: 0.0207 - val_loss: 0.0204
Epoch 7/1000
911/911 - 1s - loss: 0.0202 - val_loss: 0.0202
Epoch 8/1000
911/911 - 1s - loss: 0.0199 - val_loss: 0.0198
Epoch 9/1000
911/911 - 1s - loss: 0.0196 - val_loss: 0.0195
Epoch 10/1000
911/911 - 1s - loss: 0.0193 - val_loss: 0.0192
Epoch 11/1000
911/911 - 1s - loss: 0.0191 - val_loss: 0.0191
Epoch 12/1000
911/911 - 1s - loss: 0.0190 - val_loss: 0.0190
Epoch 13/1000
911/911 - 1s - loss: 0.0189 - val_loss: 0.0189
Epoch 14/1000
911/911 - 1s - loss

Epoch 25/1000
911/911 - 1s - loss: 0.0224 - val_loss: 0.0224
Epoch 26/1000
911/911 - 1s - loss: 0.0223 - val_loss: 0.0224
Epoch 27/1000
911/911 - 1s - loss: 0.0223 - val_loss: 0.0222
Epoch 28/1000
911/911 - 1s - loss: 0.0222 - val_loss: 0.0222
Epoch 29/1000
911/911 - 1s - loss: 0.0222 - val_loss: 0.0222
Epoch 30/1000
911/911 - 1s - loss: 0.0221 - val_loss: 0.0223
Epoch 31/1000
911/911 - 1s - loss: 0.0221 - val_loss: 0.0220
Epoch 32/1000
911/911 - 1s - loss: 0.0220 - val_loss: 0.0219
Epoch 33/1000
911/911 - 1s - loss: 0.0220 - val_loss: 0.0221
Epoch 34/1000
911/911 - 1s - loss: 0.0220 - val_loss: 0.0219
Epoch 35/1000
911/911 - 1s - loss: 0.0219 - val_loss: 0.0220
Epoch 36/1000
911/911 - 1s - loss: 0.0219 - val_loss: 0.0219
Epoch 37/1000
911/911 - 1s - loss: 0.0219 - val_loss: 0.0218
Epoch 38/1000
911/911 - 1s - loss: 0.0218 - val_loss: 0.0219
Epoch 39/1000
911/911 - 1s - loss: 0.0218 - val_loss: 0.0218
Epoch 40/1000
911/911 - 1s - loss: 0.0218 - val_loss: 0.0218
Epoch 41/1000
911/911 - 

Epoch 17/1000
911/911 - 1s - loss: 0.0268 - val_loss: 0.0267
Epoch 18/1000
911/911 - 1s - loss: 0.0267 - val_loss: 0.0267
Epoch 19/1000
911/911 - 1s - loss: 0.0266 - val_loss: 0.0266
Epoch 20/1000
911/911 - 1s - loss: 0.0265 - val_loss: 0.0265
Epoch 21/1000
911/911 - 1s - loss: 0.0265 - val_loss: 0.0264
Epoch 22/1000
911/911 - 1s - loss: 0.0264 - val_loss: 0.0263
Epoch 23/1000
911/911 - 1s - loss: 0.0263 - val_loss: 0.0267
Epoch 24/1000
911/911 - 1s - loss: 0.0262 - val_loss: 0.0263
Epoch 25/1000
911/911 - 1s - loss: 0.0262 - val_loss: 0.0261
Epoch 26/1000
911/911 - 1s - loss: 0.0261 - val_loss: 0.0261
Epoch 27/1000
911/911 - 1s - loss: 0.0261 - val_loss: 0.0264
Epoch 28/1000
911/911 - 1s - loss: 0.0260 - val_loss: 0.0260
Epoch 29/1000
911/911 - 1s - loss: 0.0260 - val_loss: 0.0261
Epoch 30/1000
911/911 - 1s - loss: 0.0259 - val_loss: 0.0258
Epoch 31/1000
911/911 - 1s - loss: 0.0259 - val_loss: 0.0261
Epoch 32/1000
911/911 - 1s - loss: 0.0258 - val_loss: 0.0258
Epoch 33/1000
911/911 - 

...

Epoch 46/1000
911/911 - 1s - loss: 0.0733 - val_loss: 0.0736
Epoch 47/1000
911/911 - 1s - loss: 0.0732 - val_loss: 0.0737
Epoch 48/1000
911/911 - 1s - loss: 0.0731 - val_loss: 0.0733
Epoch 49/1000
911/911 - 1s - loss: 0.0730 - val_loss: 0.0735
Epoch 50/1000
911/911 - 1s - loss: 0.0730 - val_loss: 0.0736
Epoch 51/1000
911/911 - 1s - loss: 0.0729 - val_loss: 0.0738
Epoch 52/1000
911/911 - 1s - loss: 0.0728 - val_loss: 0.0731
Epoch 53/1000
911/911 - 1s - loss: 0.0727 - val_loss: 0.0732
Epoch 54/1000
911/911 - 1s - loss: 0.0727 - val_loss: 0.0730
Epoch 55/1000
911/911 - 1s - loss: 0.0726 - val_loss: 0.0732
Epoch 56/1000
911/911 - 1s - loss: 0.0725 - val_loss: 0.0729
Epoch 57/1000
911/911 - 1s - loss: 0.0724 - val_loss: 0.0729
Epoch 58/1000
911/911 - 1s - loss: 0.0724 - val_loss: 0.0727
Epoch 59/1000
911/911 - 1s - loss: 0.0723 - val_loss: 0.0726
Epoch 60/1000
911/911 - 1s - loss: 0.0722 - val_loss: 0.0725
Epoch 61/1000
911/911 - 1s - loss: 0.0722 - val_loss: 0.0725
Epoch 62/1000
911/911 - 

Epoch 73/1000
911/911 - 1s - loss: 0.1013 - val_loss: 0.1022
Epoch 74/1000
911/911 - 1s - loss: 0.1012 - val_loss: 0.1016
Epoch 75/1000
911/911 - 1s - loss: 0.1011 - val_loss: 0.1023
Epoch 76/1000
911/911 - 1s - loss: 0.1011 - val_loss: 0.1015
Epoch 77/1000
911/911 - 1s - loss: 0.1010 - val_loss: 0.1018
Epoch 78/1000
911/911 - 1s - loss: 0.1010 - val_loss: 0.1017
Epoch 79/1000
911/911 - 1s - loss: 0.1009 - val_loss: 0.1015
Epoch 80/1000
911/911 - 1s - loss: 0.1009 - val_loss: 0.1018
Epoch 81/1000
911/911 - 1s - loss: 0.1008 - val_loss: 0.1016
Epoch 82/1000
911/911 - 1s - loss: 0.1008 - val_loss: 0.1016
Epoch 83/1000
911/911 - 1s - loss: 0.1008 - val_loss: 0.1015
Epoch 84/1000
911/911 - 1s - loss: 0.1007 - val_loss: 0.1022
Epoch 85/1000
911/911 - 1s - loss: 0.1007 - val_loss: 0.1012
Epoch 86/1000
911/911 - 1s - loss: 0.1006 - val_loss: 0.1017
Epoch 87/1000
911/911 - 1s - loss: 0.1005 - val_loss: 0.1011
Epoch 88/1000
911/911 - 1s - loss: 0.1006 - val_loss: 0.1016
Epoch 89/1000
911/911 - 

Epoch 69/1000
911/911 - 1s - loss: 0.1595 - val_loss: 0.1609
Epoch 70/1000
911/911 - 1s - loss: 0.1594 - val_loss: 0.1604
Epoch 71/1000
911/911 - 1s - loss: 0.1593 - val_loss: 0.1612
Epoch 72/1000
911/911 - 1s - loss: 0.1593 - val_loss: 0.1600
Epoch 73/1000
911/911 - 1s - loss: 0.1591 - val_loss: 0.1615
Epoch 74/1000
911/911 - 1s - loss: 0.1590 - val_loss: 0.1599
Epoch 75/1000
911/911 - 1s - loss: 0.1589 - val_loss: 0.1595
Epoch 76/1000
911/911 - 1s - loss: 0.1588 - val_loss: 0.1606
Epoch 77/1000
911/911 - 1s - loss: 0.1588 - val_loss: 0.1599
Epoch 78/1000
911/911 - 1s - loss: 0.1587 - val_loss: 0.1601
Epoch 79/1000
911/911 - 1s - loss: 0.1586 - val_loss: 0.1599
Epoch 80/1000
911/911 - 1s - loss: 0.1585 - val_loss: 0.1601
Epoch 81/1000
911/911 - 1s - loss: 0.1584 - val_loss: 0.1601
Epoch 82/1000
911/911 - 1s - loss: 0.1583 - val_loss: 0.1595
Epoch 83/1000
911/911 - 1s - loss: 0.1583 - val_loss: 0.1598
Epoch 84/1000
911/911 - 1s - loss: 0.1581 - val_loss: 0.1598
Epoch 85/1000
911/911 - 

Epoch 13/1000
911/911 - 1s - loss: 0.3120 - val_loss: 0.3140
Epoch 14/1000
911/911 - 1s - loss: 0.3108 - val_loss: 0.3125
Epoch 15/1000
911/911 - 1s - loss: 0.3097 - val_loss: 0.3106
Epoch 16/1000
911/911 - 1s - loss: 0.3085 - val_loss: 0.3092
Epoch 17/1000
911/911 - 1s - loss: 0.3076 - val_loss: 0.3084
Epoch 18/1000
911/911 - 1s - loss: 0.3068 - val_loss: 0.3081
Epoch 19/1000
911/911 - 1s - loss: 0.3059 - val_loss: 0.3066
Epoch 20/1000
911/911 - 1s - loss: 0.3052 - val_loss: 0.3057
Epoch 21/1000
911/911 - 1s - loss: 0.3044 - val_loss: 0.3056
Epoch 22/1000
911/911 - 1s - loss: 0.3040 - val_loss: 0.3044
Epoch 23/1000
911/911 - 1s - loss: 0.3031 - val_loss: 0.3044
Epoch 24/1000
911/911 - 1s - loss: 0.3026 - val_loss: 0.3035
Epoch 25/1000
911/911 - 1s - loss: 0.3019 - val_loss: 0.3040
Epoch 26/1000
911/911 - 1s - loss: 0.3014 - val_loss: 0.3014
Epoch 27/1000
911/911 - 1s - loss: 0.3007 - val_loss: 0.3010
Epoch 28/1000
911/911 - 1s - loss: 0.3002 - val_loss: 0.3022
Epoch 29/1000
911/911 - 

Epoch 147/1000
911/911 - 1s - loss: 0.2799 - val_loss: 0.2831
Epoch 148/1000
911/911 - 1s - loss: 0.2797 - val_loss: 0.2805
Epoch 149/1000
911/911 - 1s - loss: 0.2795 - val_loss: 0.2813
Epoch 150/1000
911/911 - 1s - loss: 0.2798 - val_loss: 0.2801
Epoch 151/1000
911/911 - 1s - loss: 0.2794 - val_loss: 0.2804
Epoch 152/1000
911/911 - 1s - loss: 0.2796 - val_loss: 0.2808
Epoch 153/1000
911/911 - 1s - loss: 0.2795 - val_loss: 0.2811
Epoch 154/1000
911/911 - 1s - loss: 0.2792 - val_loss: 0.2795
Epoch 155/1000
911/911 - 1s - loss: 0.2791 - val_loss: 0.2809
Epoch 156/1000
911/911 - 1s - loss: 0.2791 - val_loss: 0.2791
Epoch 157/1000
911/911 - 1s - loss: 0.2790 - val_loss: 0.2802
Epoch 158/1000
911/911 - 1s - loss: 0.2790 - val_loss: 0.2788
Epoch 159/1000
911/911 - 1s - loss: 0.2790 - val_loss: 0.2804
Epoch 160/1000
911/911 - 1s - loss: 0.2787 - val_loss: 0.2790
Epoch 161/1000
911/911 - 1s - loss: 0.2787 - val_loss: 0.2791
Epoch 162/1000
911/911 - 1s - loss: 0.2786 - val_loss: 0.2797
Epoch 16

## 模型评估

In [4]:
# 评估函数定义
# 异常相关系数
import math
def acc(actual, predicted):
    pred_avg = np.average(predicted)

    act_avg = np.average(actual)
    diff_pred = predicted - pred_avg
    diff_act = actual - act_avg
    numerator = np.mean(np.sum(diff_pred*diff_act, axis=0))
    denominator = math.sqrt(np.mean(np.sum(diff_pred**2, axis=0)) * np.mean(np.sum(diff_act**2, axis=0)))
    ret_val = numerator/denominator
    return (100 * ret_val)


# 查看损失
def look_loss(history):
    plt.figure(figsize=(6.3, 2.5), dpi = 100)
    plt.plot(history.history['loss'], label='training data')
    plt.plot(history.history['val_loss'], label='validation data')
    plt.title('Loss')
    plt.ylabel('MSE Loss')
    plt.xlabel('Number of epochs')
    plt.grid()
    plt.legend(loc="upper right")
    plt.show()


# 模型预测
def all_estimate(best_model,x_test,y_test):
    testPred_1 = best_model.predict(x_test)
    a = y_test
    y_test_p = a.reshape(-1,1)
    testPred_p = testPred_1.reshape(-1,1)


    rmse = sqrt(mean_squared_error(y_test_p,testPred_p))
    print('Test RMSE: %.3f' % rmse)
    #print('%.3f' % rmse)
    #print('acc:',acc(y_test_p,testPred_p))

    r2 = r2_score(y_test_p,testPred_p)
    print("R² score: %.4f" %r2)  
    #print("%.4f" % r2)
    

# def respective_estimate(best_model,x_test,y_test):
#     re2_list = []
#     rmse_list = []
#     for i in range(6):
#         # 模型预测
#         testPred = best_model.predict(x_test[i:i+1])
#         a = y_test[i:i+1]
#         y_test_p = a.reshape(-1,1)
#         testPred_p = testPred.reshape(-1,1)
#         r2 = r2_score(y_test_p,testPred_p)
#         rmse = sqrt(mean_squared_error(y_test_p,testPred_p))
#         re2_list.append(r2)
#         rmse_list.append(rmse)    
#     return re2_list,rmse_list

In [5]:
#look_loss(history)

In [6]:
# 模型预测  
depth=[5,10,15,20,30,40,50,60,70,80,90,100,125,150,175,200,225,250,275,300,350,400,500,600,700,800,900,1000,1100,1300,1500,1750,2000]
result = [] 
for i in range(0,33):
    #best_model = tf.keras.models.load_model('./model/BPNN/sali/bpnn_model_'+str(depth[i])+'m_2.h5') 
    best_model = tf.keras.models.load_model('../model/new_0-2000m_24.6.6/BPNN/temp/model1/bpnn_temp_'+str(depth[i])+'m.h5')
    print('预测第'+str(depth[i])+'m的温度')
    all_estimate(best_model,x_test,y_test[:,i:i+1])
    testPred = best_model.predict(x_test)
    result.append(testPred)

预测第5m的温度


NameError: name 'x_test' is not defined

In [ ]:
# 到达这里

# 预测结果可视化

## 为归一化之后的预测值填充nan

In [7]:
# 查看预测结果（testPred）与原始数据的形状
testPred.shape,test_flatten.shape 

NameError: name 'testPred' is not defined

In [8]:
nan_mask = np.isnan(test_flatten) #返回一个与 test_flatten 形状相同的布尔数组，其中对应的元素是 NaN 时为 True，否则为 False
nan_masks = nan_mask.any(axis=1) # true 代表通道中有nan   
print(nan_masks.shape)

NameError: name 'test_flatten' is not defined

In [9]:
### 创建一个形状与nan_masks相同且全是nan的数组
testPred_with_nan = np.full(((nan_masks.shape[0]),), np.nan)  

NameError: name 'nan_masks' is not defined

In [10]:
num = 0 
for i,nan_mask in enumerate(nan_masks):   # i从0开始

    if (nan_mask == False): # False 代表没有nan，使用预测值进行填充
        testPred_with_nan[i] = testPred[num]
        num = num + 1
        
print(num)

NameError: name 'nan_masks' is not defined

## 为归一化之后的真实值填充nan 

In [11]:
print(nan_masks.shape)

NameError: name 'nan_masks' is not defined

In [12]:
y_test_with_nan = np.full(((nan_masks.shape[0]),), np.nan) 

NameError: name 'nan_masks' is not defined

In [13]:
y_test[:,14:15].shape 

NameError: name 'y_test' is not defined

In [14]:
num = 0 
for i,nan_mask in enumerate(nan_masks):   # i从0开始

    if (nan_mask == False): # False 代表没有nan，使用预测值进行填充
        y_test_with_nan[i] = y_test[:,14:][num]  # 14代表第14层，应根据具体的层数进行替换。
        num = num + 1
        
print(num)

NameError: name 'nan_masks' is not defined

# 结果可视化

In [18]:
testPred_with_nan_reshape  = testPred_with_nan.reshape((12,280,600))
y_test_with_nan_reshape = y_test_with_nan.reshape((12,280,600))
testPred_with_nan_reshape.shape , y_test_with_nan_reshape.shape 

NameError: name 'testPred_with_nan' is not defined

In [16]:
lat = []
lon = []
time = []

In [ ]:
lon.append(data_ssh['lon'][:].data)
lat.append(data_ssh['lat'][:].data)

In [ ]:
start_year = 2015
end_year =2015
time = []
date_range = pd.date_range(datetime(start_year,1,1),datetime(end_year+1,1,1),freq='1M')
len(date_range)
for i in range(len(date_range)):
    time.append(i)

In [ ]:
new_NC = nc.Dataset("result1000m.nc", 'w', format='NETCDF4')

'''
定义维度，后一个参数表示维度的长度，因为是合并的同一个产品的数据，所以是统一
的,注意维度的长度一定要和读入的数据匹配
'''

new_NC.createDimension('lat', len(lat[0]))
new_NC.createDimension('lon', len(lon[0]))
new_NC.createDimension('true_sta', len(y_test_with_nan_reshape))
new_NC.createDimension('pred_sta', len(testPred_with_nan_reshape))

#new_NC.createDimension('level', 20)
new_NC.createDimension('time', len(time))




new_NC.createVariable('lat', 'f', ("lat"))
new_NC.createVariable('lon', 'f', ("lon"))
#new_NC.createVariable('level', 'f', ("level"))
new_NC.createVariable('true_sta', 'f',("time","lat","lon"))
new_NC.createVariable('pred_sta', 'f',("time","lat","lon"))

time_var = new_NC.createVariable('time', 'f4',("time"))
time_var.units = 'months since 2015-01-15'
time_var.long_name = 'Months in Monthly Means'
time_var.axis = 'T'



#向变量中填充数据
new_NC.variables['lat'][:] = lat[0]
new_NC.variables['lon'][:] = lon[0]
new_NC.variables['time'][:] = np.array(time)
#new_NC.variables['level'][:] = level

new_NC.variables['true_sta'][:]=np.array(y_test_with_nan_reshape)
new_NC.variables['pred_sta'][:]=np.array(testPred_with_nan_reshape)


#最后记得关闭文件
new_NC.close()
